**TXT to Json String**

In [42]:
import os
import json
import re

In [43]:
PATH_dir = '../data/en_ner_data'
file_name_en_train = 'train.txt'
file_name_en_test = 'test.txt'
file_name_en_valid = 'valid.txt'

In [44]:
file_name = file_name_en_valid
PATH_data = os.path.join(PATH_dir, file_name)

In [45]:
pattern = re.compile('[IB]-[a-zA-Z]+')


In [46]:

with open(PATH_data, 'r') as f:
	
	fdata = f.read().split('\n\n')
	data = []

	for sentences in fdata[:-1]:

		sdata = []

		morph_sent = ""
		word_list = []
		ner_list = []

		sentence = sentences.split('\n')
		
		if sentence[-1] == "":
			sentence.pop()

		for line in sentence:
			components = line.split(' ')
			word = components[0].split("/")[0]

			if not (re.match(pattern, components[-1]) or components[-1] == 'O'):
				continue

			morph_sent += word + " "
			word_list.append(word)
			ner_list.append(components[-1])
		
		morph_sent = morph_sent[:-1]

		sdata.append(morph_sent)
		sdata.append(word_list)
		sdata.append(ner_list)
		
		data.append(sdata)

		
		
		

In [47]:

for sdata in data[:5]:
	print("morph_sent", sdata[0], "words", sdata[1], "ner", sdata[2], sep = '\n', end = '\n\n')

morph_sent
-DOCSTART-
words
['-DOCSTART-']
ner
['O']

morph_sent
CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
words
['CRICKET', '-', 'LEICESTERSHIRE', 'TAKE', 'OVER', 'AT', 'TOP', 'AFTER', 'INNINGS', 'VICTORY', '.']
ner
['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

morph_sent
LONDON 1996-08-30
words
['LONDON', '1996-08-30']
ner
['B-LOC', 'O']

morph_sent
West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .
words
['West', 'Indian', 'all-rounder', 'Phil', 'Simmons', 'took', 'four', 'for', '38', 'on', 'Friday', 'as', 'Leicestershire', 'beat', 'Somerset', 'by', 'an', 'innings', 'and', '39', 'runs', 'in', 'two', 'days', 'to', 'take', 'over', 'at', 'the', 'head', 'of', 'the', 'county', 'championship', '.']
ner
['B-MISC', 'I-MISC', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'B-ORG', 'O', 'O', 'O'

**Tokenizing Json String**

In [48]:

from transformers import BertTokenizer

In [49]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)

In [50]:
preprodata = []

for sdata in data:
	morph_to_tokens = tokenizer.tokenize(sdata[0])

	prepro_sdata = []
	tokenized_ner_list = []
	iterated_word = iter(sdata[1])
	iterated_ner = iter(sdata[2])

	word = ""
	entire_word = next(iterated_word)
	ner = next(iterated_ner)
	tokenized_ner_list.append(ner)

	for token in morph_to_tokens:

		if token[:2] == "##":
			word += token[2:]
		else:
			word += token
		
		if word == (entire_word or "[UNK]") and word != sdata[1][-1]:
			word = ""
			entire_word = next(iterated_word)
			ner = next(iterated_ner)
			tokenized_ner_list.append(ner)

		else:
			if ner == "O":
				tokenized_ner_list.append("O")
			else:
				tokenized_ner_list.append("I" + ner[1:])

	prepro_sdata.append(sdata[0])
	prepro_sdata.append(morph_to_tokens)
	prepro_sdata.append(tokenized_ner_list)

	
	preprodata.append(prepro_sdata)
	

In [51]:
print(preprodata[0][0])
print(preprodata[0][1])
print(preprodata[0][2])
print("\n")
print(preprodata[1][0])
print(preprodata[1][1])
print(preprodata[1][2])
print("\n")
print(preprodata[191][0])
print(preprodata[191][1])
print(preprodata[191][2])

-DOCSTART-
['-', 'DO', '##CS', '##TA', '##RT', '-']
['O', 'O', 'O', 'O', 'O', 'O', 'O']


CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
['CR', '##IC', '##K', '##ET', '-', 'LE', '##IC', '##ES', '##TER', '##SH', '##IR', '##E', 'TA', '##K', '##E', 'O', '##VE', '##R', 'AT', 'TOP', 'AF', '##TER', 'IN', '##NI', '##NG', '##S', 'VI', '##CT', '##OR', '##Y', '.']
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


Men 's singles , second round
['Men', "'", 's', 'singles', ',', 'second', 'round']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [52]:
prepro_file_name = 'prepro_' + file_name.rsplit('.')[0]
PATH_preprodata = os.path.join(PATH_dir, prepro_file_name + '.json')

with open(PATH_preprodata, 'w') as f:
    json.dump(preprodata, f, ensure_ascii=False, indent='\t')

**Tag List**

In [53]:
prepro_tag_list = []
PATH_preprodata_tag_list = os.path.join(PATH_dir, prepro_file_name + '_tag_list.json')

with open(PATH_preprodata_tag_list, 'w') as f:

	for prepro_sdata in preprodata:
		prepro_tag_list.extend(prepro_sdata[2])
	
	prepro_tag_list = list(set(prepro_tag_list))

	json.dump(prepro_tag_list, f, ensure_ascii=False, indent='\t')


In [54]:

# prepro_tag_cnt_dict = {}

# for tag in prepro_tag_extend_list:
# 	try: prepro_tag_cnt_dict[tag] += 1
# 	except: prepro_tag_cnt_dict[tag] = 1